# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare and clean the data

In [9]:
# Notes:
# OSM road network is in WGS84. Projected each dataset to match.
# Starting as CSV (dataframe) and deriving geometry from there tends to avoid read errors.

#### Previously: created polygon shapefile from MapSPAM summary value of all crop areas and point shapefile from UMD land cover cropland class.
#### Now, joining them together and dealing with spatial gaps so that all MapSPAM values have a point of origin for the graph.

#### Pseudocode:
    4. agriculture.shp = Spatial join SPAM["val"] and SPAM["SID"] onto lc.shp (full outer join)
    5. Where ["SID"] = NULL, ["val"] = 0
    6. Where ["lc_crop"] = NULL:
        SPAM_centroid.shp = SPAM.shp to centroid (retain ["val"] field)
        SPAM_cent_subset.shp = Select SPAM_centroid based on lc_crop = NULL and export
        Append agriculture.shp with SPAM_cent_subset.shp
    7. ["lc_sum"] = Sum ["lcID"] per ["SID"]
    8. Divide SPAM value by lc_sum and assign it to landcover["spam_V"]

In [10]:
spam = gpd.read_file("R:\SEN\GEO\Team\Projects\SEN_TransportOV\SEN_TransportOV.gdb", layer="spam2017")
spam.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2511 entries, 0 to 2510
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Id            2511 non-null   int64   
 1   gridcode      2511 non-null   int64   
 2   Shape_Length  2511 non-null   float64 
 3   Shape_Area    2511 non-null   float64 
 4   geometry      2511 non-null   geometry
dtypes: float64(2), geometry(1), int64(2)
memory usage: 98.2 KB


In [11]:
spam

,Id,gridcode,Shape_Length,Shape_Area,geometry
0,1,129669,0.314403,0.004755,"MULTIPOLYGON (((-11.66734 16.91696, -11.76894 ..."
1,2,70993,0.333332,0.006944,"MULTIPOLYGON (((-15.33399 16.83363, -15.41733 ..."
2,3,87412,0.333332,0.006944,"MULTIPOLYGON (((-15.25066 16.83363, -15.33399 ..."
3,4,598,0.314403,0.004755,"MULTIPOLYGON (((-13.83400 16.83363, -13.93560 ..."
4,5,54822,0.333332,0.006944,"MULTIPOLYGON (((-15.33399 16.75029, -15.41733 ..."
...,...,...,...,...,...
2506,2507,411859,0.333332,0.006944,"MULTIPOLYGON (((-11.41734 12.00031, -11.50067 ..."
2507,2508,497200,0.333332,0.006944,"MULTIPOLYGON (((-11.33401 12.00031, -11.41734 ..."
2508,2509,217938,0.333332,0.006944,"MULTIPOLYGON (((-11.25067 12.00031, -11.33401 ..."
2509,2510,201222,0.333332,0.006944,"MULTIPOLYGON (((-11.16734 12.00031, -11.25067 ..."


Was having trouble taking the land cover layer from a raster to point vector in Arc Pro. I trust R better than Python for raster data, so I ran the following code to get a CSV with point coordinates:

library(raster)
library(sp)
library(rgdal)
library(sf)
library(dplyr)

wd <- getwd()
landcover <- raster(paste0(wd, "/UMD_senegambia_crop.tif"))

lc_pt <- rasterToPoints(landcover, fun=NULL, spatial=FALSE)
lc_pt_df <- as.data.frame(lc_pt)


lc_pt_df$index <- 1:nrow(lc_pt_df)
lc_pt_df <- rename(lc_pt_df, ID_lc = index)

write.csv(lc_pt_df, file = "lc_pt.csv")

In [12]:
lc = os.path.join(pth, "lc_pt.csv")
lc = pd.read_csv(lc)
crs = {'init': 'epsg:4326'} 
geometry = [Point(xy) for xy in zip(lc.x, lc.y)]
lc = GeoDataFrame(lc, crs=crs, geometry=geometry) 
lc.info()
# The csv is 320 mb and 7654999 rows. It will take a few minutes to georeference.

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7654999 entries, 0 to 7654998
Data columns (total 6 columns):
 #   Column               Dtype   
---  ------               -----   
 0   Unnamed: 0           int64   
 1   x                    float64 
 2   y                    float64 
 3   UMD_senegambia_crop  int64   
 4   ID_lc                int64   
 5   geometry             geometry
dtypes: float64(2), geometry(1), int64(3)
memory usage: 350.4 MB


In [13]:
lc

,Unnamed: 0,x,y,UMD_senegambia_crop,ID_lc,geometry
0,1,-15.990057,16.693707,17,1,POINT (-15.99006 16.69371)
1,2,-15.989787,16.693707,17,2,POINT (-15.98979 16.69371)
2,3,-15.989518,16.693707,17,3,POINT (-15.98952 16.69371)
3,4,-15.925378,16.693707,17,4,POINT (-15.92538 16.69371)
4,5,-15.925109,16.693707,17,5,POINT (-15.92511 16.69371)
...,...,...,...,...,...,...
7654994,7654995,-15.940200,15.171060,17,7654995,POINT (-15.94020 15.17106)
7654995,7654996,-15.939930,15.171060,17,7654996,POINT (-15.93993 15.17106)
7654996,7654997,-15.939660,15.171060,17,7654997,POINT (-15.93966 15.17106)
7654997,7654998,-15.939390,15.171060,17,7654998,POINT (-15.93939 15.17106)


#### Continue with SPAM and land cover join.

In [14]:
spam = spam.to_crs("EPSG:31028")
lc = lc.to_crs("EPSG:31028")
spam.crs == lc.crs

True

In [15]:
spam.head()

,Id,gridcode,Shape_Length,Shape_Area,geometry
0,1,129669,0.314403,0.004755,"MULTIPOLYGON (((854888.627 1873298.371, 844002..."
1,2,70993,0.333332,0.006944,"MULTIPOLYGON (((464241.542 1861102.381, 455363..."
2,3,87412,0.333332,0.006944,"MULTIPOLYGON (((473119.710 1861089.263, 464241..."
3,4,598,0.314403,0.004755,"MULTIPOLYGON (((624054.169 1861438.466, 613211..."
4,5,54822,0.333332,0.006944,"MULTIPOLYGON (((464226.016 1851883.675, 455343..."


In [16]:
lc.head()

,Unnamed: 0,x,y,UMD_senegambia_crop,ID_lc,geometry
0,1,-15.990057,16.693707,17,1,POINT (394264.774 1845856.132)
1,2,-15.989787,16.693707,17,2,POINT (394293.510 1845855.989)
2,3,-15.989518,16.693707,17,3,POINT (394322.246 1845855.846)
3,4,-15.925378,16.693707,17,4,POINT (401161.317 1845823.013)
4,5,-15.925109,16.693707,17,5,POINT (401190.052 1845822.879)


In [17]:
spam.rename(columns={'Id':'ID_spam', 'gridcode':'val'}, inplace=True)
spam = spam.drop(columns=['Shape_Length', 'Shape_Area'])
lc = lc.drop(columns=['Unnamed: 0', 'UMD_senegambia_crop']) 

In [18]:
# This is the main dataset to be used for the network analysis.
agriculture = gpd.sjoin(lc, spam, how='left', lsuffix='_lc', rsuffix='_spam')
agriculture['val'] = agriculture['val'].fillna(1) # Assign a value of one to non-matches.
agriculture

,x,y,ID_lc,geometry,index__spam,ID_spam,val
0,-15.990057,16.693707,1,POINT (394264.774 1845856.132),NaN,NaN,1.0
1,-15.989787,16.693707,2,POINT (394293.510 1845855.989),NaN,NaN,1.0
2,-15.989518,16.693707,3,POINT (394322.246 1845855.846),NaN,NaN,1.0
3,-15.925378,16.693707,4,POINT (401161.317 1845823.013),NaN,NaN,1.0
4,-15.925109,16.693707,5,POINT (401190.052 1845822.879),NaN,NaN,1.0
...,...,...,...,...,...,...,...
7654994,-15.940200,15.171060,7654995,POINT (398822.078 1677384.290),458.0,459.0,670742.0
7654995,-15.939930,15.171060,7654996,POINT (398851.085 1677384.166),458.0,459.0,670742.0
7654996,-15.939660,15.171060,7654997,POINT (398880.092 1677384.041),458.0,459.0,670742.0
7654997,-15.939390,15.171060,7654998,POINT (398909.100 1677383.917),458.0,459.0,670742.0


#### Land cover that doesn't have a SPAM grid was assigned a value of 1. Now need to provide points for SPAM data where there is no land cover pixel.
#### Create xy fields from spam centroids now for later.  Need WGS84 points for all SPAM grids that don't have any land cover points to map to.

In [19]:
spam_xy = spam.to_crs("EPSG:31028") # Geographic CRSs can cause incorrect centroid measures. Use projected.
spam_xy['x'] = spam_xy.centroid.x
spam_xy['y'] = spam_xy.centroid.y
spam_xy.to_csv(os.path.join(out_pth, 'spam_xy.csv'))

In [20]:
# Create new point dataframe from centroids.
spam_xy = os.path.join(out_pth, "spam_xy.csv")
spam_xy = pd.read_csv(spam_xy)
geometry = [Point(xy) for xy in zip(spam_xy.x, spam_xy.y)]
crs = "EPSG:31028"
spam_xy = GeoDataFrame(spam_xy, crs=crs, geometry=geometry) 
spam_xy

,Unnamed: 0,ID_spam,val,geometry,x,y
0,0,1,129669,POINT (850182.549 1878026.114),850182.549397,1.878026e+06
1,1,2,70993,POINT (459811.192 1865719.882),459811.192338,1.865720e+06
2,2,3,87412,POINT (468687.447 1865704.861),468687.447370,1.865705e+06
3,3,4,598,POINT (619404.953 1866211.462),619404.953257,1.866211e+06
4,4,5,54822,POINT (459793.687 1856501.104),459793.687462,1.856501e+06
...,...,...,...,...,...,...
2506,2506,2507,411859,POINT (885461.910 1333603.406),885461.910220,1.333603e+06
2507,2507,2508,497200,POINT (894548.438 1333722.104),894548.438017,1.333722e+06
2508,2508,2509,217938,POINT (903635.735 1333843.581),903635.735036,1.333844e+06
2509,2509,2510,201222,POINT (912723.819 1333967.838),912723.819020,1.333968e+06


In [21]:
# But we need the xy fields to be in WGS84 for snapping to the graph and matching origins table, so we re-project again.
spam_xy = spam_xy.to_crs("EPSG:4326")
spam_xy["geom_WGS84"] = spam_xy["geometry"].astype('str')

# Strip the geometry column of extra characters
spam_xy["geom_WGS84"] = spam_xy["geom_WGS84"].str.strip('POINT ') 
spam_xy["geom_WGS84"] = spam_xy["geom_WGS84"].str.strip('()')

# 3. Split the column into two based on the space between the coordinates.
XY_spam = spam_xy["geom_WGS84"].str.split(" ", expand=True)
spam_xy["X"] = XY_spam[0]
spam_xy["Y"] = XY_spam[1]

spam_xy.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2511 entries, 0 to 2510
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  2511 non-null   int64   
 1   ID_spam     2511 non-null   int64   
 2   val         2511 non-null   int64   
 3   geometry    2511 non-null   geometry
 4   x           2511 non-null   float64 
 5   y           2511 non-null   float64 
 6   geom_WGS84  2511 non-null   object  
 7   X           2511 non-null   object  
 8   Y           2511 non-null   object  
dtypes: float64(2), geometry(1), int64(3), object(3)
memory usage: 176.7+ KB


In [22]:
spam_xy['X'] = spam_xy['X'].astype(float)
spam_xy['Y'] = spam_xy['Y'].astype(float)
spam_xy.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2511 entries, 0 to 2510
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  2511 non-null   int64   
 1   ID_spam     2511 non-null   int64   
 2   val         2511 non-null   int64   
 3   geometry    2511 non-null   geometry
 4   x           2511 non-null   float64 
 5   y           2511 non-null   float64 
 6   geom_WGS84  2511 non-null   object  
 7   X           2511 non-null   float64 
 8   Y           2511 non-null   float64 
dtypes: float64(4), geometry(1), int64(3), object(1)
memory usage: 176.7+ KB


In [23]:
spam_xy

,Unnamed: 0,ID_spam,val,geometry,x,y,geom_WGS84,X,Y
0,0,1,129669,POINT (-11.71072 16.96034),850182.549397,1.878026e+06,-11.710716 16.96034,-11.710716,16.960340
1,1,2,70993,POINT (-15.37566 16.87529),459811.192338,1.865720e+06,-15.375659 16.875294,-15.375659,16.875294
2,2,3,87412,POINT (-15.29233 16.87529),468687.447370,1.865705e+06,-15.292326 16.875294,-15.292326,16.875294
3,3,4,598,POINT (-13.87737 16.87701),619404.953257,1.866211e+06,-13.877374 16.877007,-13.877374,16.877007
4,4,5,54822,POINT (-15.37566 16.79196),459793.687462,1.856501e+06,-15.375659 16.791961,-15.375659,16.791961
...,...,...,...,...,...,...,...,...,...
2506,2506,2507,411859,POINT (-11.45901 12.04198),885461.910220,1.333603e+06,-11.459007 12.04198,-11.459007,12.041980
2507,2507,2508,497200,POINT (-11.37567 12.04198),894548.438017,1.333722e+06,-11.375674 12.04198,-11.375674,12.041980
2508,2508,2509,217938,POINT (-11.29234 12.04198),903635.735036,1.333844e+06,-11.292341 12.04198,-11.292341,12.041980
2509,2509,2510,201222,POINT (-11.20901 12.04198),912723.819020,1.333968e+06,-11.209008 12.04198,-11.209008,12.041980


#### This is how we still retain SPAM values where there was no land cover cell to represent it. 
#### There may be a more efficient method here. sjoin does not allow full outer joins.

In [24]:
# Find which SPAM cells were not joined to land cover, create a point for them, and append them to the origins (agriculture) dataset.
spam_nomatch = spam[~spam.ID_spam.isin(agriculture.ID_spam)]
spam_nomatch

,ID_spam,val,geometry
0,1,129669,"MULTIPOLYGON (((854888.627 1873298.371, 844002..."
1,2,70993,"MULTIPOLYGON (((464241.542 1861102.381, 455363..."
2,3,87412,"MULTIPOLYGON (((473119.710 1861089.263, 464241..."
3,4,598,"MULTIPOLYGON (((624054.169 1861438.466, 613211..."
4,5,54822,"MULTIPOLYGON (((464226.016 1851883.675, 455343..."
...,...,...,...
2506,2507,411859,"MULTIPOLYGON (((890065.210 1329046.565, 880977..."
2507,2508,497200,"MULTIPOLYGON (((899153.528 1329166.261, 890065..."
2508,2509,217938,"MULTIPOLYGON (((908242.625 1329288.728, 899153..."
2509,2510,201222,"MULTIPOLYGON (((917332.518 1329413.967, 908242..."


In [25]:
spam_xy.rename(columns={'Id':'ID_spam', 'gridcode':'val'}, inplace=True) # To match the join field of spam_nomatch
spam_nomatch = pd.merge(spam_nomatch, spam_xy, on='ID_spam',how='left') # Add the WGS84 xy fields created earlier only for SPAM grids that didn't have land cover points.
spam_nomatch
# Resulting dataset should have same number of rows as earlier spam_nomatch table.

,ID_spam,val_x,geometry_x,Unnamed: 0,val_y,geometry_y,x,y,geom_WGS84,X,Y
0,1,129669,"MULTIPOLYGON (((854888.627 1873298.371, 844002...",0,129669,POINT (-11.71072 16.96034),850182.549397,1.878026e+06,-11.710716 16.96034,-11.710716,16.960340
1,2,70993,"MULTIPOLYGON (((464241.542 1861102.381, 455363...",1,70993,POINT (-15.37566 16.87529),459811.192338,1.865720e+06,-15.375659 16.875294,-15.375659,16.875294
2,3,87412,"MULTIPOLYGON (((473119.710 1861089.263, 464241...",2,87412,POINT (-15.29233 16.87529),468687.447370,1.865705e+06,-15.292326 16.875294,-15.292326,16.875294
3,4,598,"MULTIPOLYGON (((624054.169 1861438.466, 613211...",3,598,POINT (-13.87737 16.87701),619404.953257,1.866211e+06,-13.877374 16.877007,-13.877374,16.877007
4,5,54822,"MULTIPOLYGON (((464226.016 1851883.675, 455343...",4,54822,POINT (-15.37566 16.79196),459793.687462,1.856501e+06,-15.375659 16.791961,-15.375659,16.791961
...,...,...,...,...,...,...,...,...,...,...,...
2061,2507,411859,"MULTIPOLYGON (((890065.210 1329046.565, 880977...",2506,411859,POINT (-11.45901 12.04198),885461.910220,1.333603e+06,-11.459007 12.04198,-11.459007,12.041980
2062,2508,497200,"MULTIPOLYGON (((899153.528 1329166.261, 890065...",2507,497200,POINT (-11.37567 12.04198),894548.438017,1.333722e+06,-11.375674 12.04198,-11.375674,12.041980
2063,2509,217938,"MULTIPOLYGON (((908242.625 1329288.728, 899153...",2508,217938,POINT (-11.29234 12.04198),903635.735036,1.333844e+06,-11.292341 12.04198,-11.292341,12.041980
2064,2510,201222,"MULTIPOLYGON (((917332.518 1329413.967, 908242...",2509,201222,POINT (-11.20901 12.04198),912723.819020,1.333968e+06,-11.209008 12.04198,-11.209008,12.041980


In [26]:
spam_nomatch.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID_spam     2066 non-null   int64   
 1   val_x       2066 non-null   int64   
 2   geometry_x  2066 non-null   geometry
 3   Unnamed: 0  2066 non-null   int64   
 4   val_y       2066 non-null   int64   
 5   geometry_y  2066 non-null   geometry
 6   x           2066 non-null   float64 
 7   y           2066 non-null   float64 
 8   geom_WGS84  2066 non-null   object  
 9   X           2066 non-null   float64 
 10  Y           2066 non-null   float64 
dtypes: float64(4), geometry(2), int64(4), object(1)
memory usage: 193.7+ KB


In [28]:
# Clean up the no match dataset prior to appending to final origins file. Make sure XY fields are exact matches, including case.
spam_nomatch = spam_nomatch.drop(columns=['Unnamed: 0', 'val_y', 'geom_WGS84', 'x', 'y'])
spam_nomatch.rename(columns={'val_x':'val', 'geometry_x':'geom_UTM', 'geometry_y':'geom_WGS84', 'X':'x', 'Y':'y'}, inplace=True)
spam_nomatch.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID_spam     2066 non-null   int64   
 1   val         2066 non-null   int64   
 2   geom_UTM    2066 non-null   geometry
 3   geom_WGS84  2066 non-null   geometry
 4   x           2066 non-null   float64 
 5   y           2066 non-null   float64 
dtypes: float64(2), geometry(2), int64(2)
memory usage: 113.0 KB


In [29]:
# Just in case, I want to reload the no matches as a regular dataframe and make sure all data types are appropriate.
spam_nomatch.to_csv(os.path.join(out_pth, 'spam_nomatch.csv'))
spam_nomatch = os.path.join(out_pth, "spam_nomatch.csv")
spam_nomatch = pd.read_csv(spam_nomatch)
spam_nomatch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2066 entries, 0 to 2065
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2066 non-null   int64  
 1   ID_spam     2066 non-null   int64  
 2   val         2066 non-null   int64  
 3   geom_UTM    2066 non-null   object 
 4   geom_WGS84  2066 non-null   object 
 5   x           2066 non-null   float64
 6   y           2066 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 113.1+ KB


In [30]:
agriculture = agriculture.append(spam_nomatch) 
agriculture
# Make sure XY fields appended onto the agriculture XY fields.

,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0,geom_UTM,geom_WGS84
0,-15.990057,16.693707,1.0,POINT (394264.774 1845856.132),NaN,NaN,1.0,NaN,NaN,NaN
1,-15.989787,16.693707,2.0,POINT (394293.510 1845855.989),NaN,NaN,1.0,NaN,NaN,NaN
2,-15.989518,16.693707,3.0,POINT (394322.246 1845855.846),NaN,NaN,1.0,NaN,NaN,NaN
3,-15.925378,16.693707,4.0,POINT (401161.317 1845823.013),NaN,NaN,1.0,NaN,NaN,NaN
4,-15.925109,16.693707,5.0,POINT (401190.052 1845822.879),NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2061,-11.459007,12.041980,NaN,None,NaN,2507.0,411859.0,2061.0,MULTIPOLYGON (((890065.2102591199 1329046.5648...,POINT (-11.45900694020285 12.041979526954448)
2062,-11.375674,12.041980,NaN,None,NaN,2508.0,497200.0,2062.0,MULTIPOLYGON (((899153.5281643758 1329166.2613...,POINT (-11.375673926651558 12.041979526573865)
2063,-11.292341,12.041980,NaN,None,NaN,2509.0,217938.0,2063.0,MULTIPOLYGON (((908242.6248140114 1329288.7282...,POINT (-11.292340913098172 12.041979526183944)
2064,-11.209008,12.041980,NaN,None,NaN,2510.0,201222.0,2064.0,MULTIPOLYGON (((917332.5179180377 1329413.9667...,POINT (-11.209007899992315 12.041979525784605)


In [35]:
agriculture.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7657065 entries, 0 to 2065
Data columns (total 8 columns):
 #   Column       Dtype   
---  ------       -----   
 0   x            float64 
 1   y            float64 
 2   ID_lc        float64 
 3   geometry     geometry
 4   index__spam  float64 
 5   ID_spam      float64 
 6   val          float64 
 7   Unnamed: 0   float64 
dtypes: float64(7), geometry(1)
memory usage: 525.8 MB


In [34]:
agriculture = agriculture.drop(columns=['geom_UTM', 'geom_WGS84'])
agriculture['x'] = agriculture['x'].astype(float) # Just in case.
agriculture['y'] = agriculture['y'].astype(float)
agriculture['ID_lc'] = agriculture['ID_lc'].astype(float)
agriculture['ID_spam'] = agriculture['ID_spam'].astype(float)
agriculture['val'] = agriculture['val'].astype(float)

In [36]:
# Give a dummy ID for SPAM locations that don't have land cover pixels, and vice versa.
agriculture['ID_lc'] = agriculture['ID_lc'].fillna(0.1)
agriculture['ID_spam'] = agriculture['ID_spam'].fillna(0.1)

In [37]:
# Split SPAM values evenly among the land cover cells within each SPAM cell.
agriculture['lc_count'] = 0
agriculture['lc_count'] = agriculture['ID_lc'].groupby(agriculture['ID_spam']).transform('count') # Counts number of land cover cells grouped in a given SPAM cell.
agriculture['spam_V'] = 0
agriculture['spam_V'] = agriculture['val'] / agriculture['lc_count'] # Divides SPAM value evenly among the land cover cells within that SPAM cell.
agriculture

,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0,lc_count,spam_V
0,-15.990057,16.693707,1.0,POINT (394264.774 1845856.132),NaN,0.1,1.0,NaN,574130,0.000002
1,-15.989787,16.693707,2.0,POINT (394293.510 1845855.989),NaN,0.1,1.0,NaN,574130,0.000002
2,-15.989518,16.693707,3.0,POINT (394322.246 1845855.846),NaN,0.1,1.0,NaN,574130,0.000002
3,-15.925378,16.693707,4.0,POINT (401161.317 1845823.013),NaN,0.1,1.0,NaN,574130,0.000002
4,-15.925109,16.693707,5.0,POINT (401190.052 1845822.879),NaN,0.1,1.0,NaN,574130,0.000002
...,...,...,...,...,...,...,...,...,...,...
2061,-11.459007,12.041980,0.1,None,NaN,2507.0,411859.0,2061.0,1,411859.000000
2062,-11.375674,12.041980,0.1,None,NaN,2508.0,497200.0,2062.0,1,497200.000000
2063,-11.292341,12.041980,0.1,None,NaN,2509.0,217938.0,2063.0,1,217938.000000
2064,-11.209008,12.041980,0.1,None,NaN,2510.0,201222.0,2064.0,1,201222.000000


In [38]:
agriculture.head(40)

,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0,lc_count,spam_V
0,-15.990057,16.693707,1.0,POINT (394264.774 1845856.132),NaN,0.1,1.0,NaN,574130,0.000002
1,-15.989787,16.693707,2.0,POINT (394293.510 1845855.989),NaN,0.1,1.0,NaN,574130,0.000002
2,-15.989518,16.693707,3.0,POINT (394322.246 1845855.846),NaN,0.1,1.0,NaN,574130,0.000002
3,-15.925378,16.693707,4.0,POINT (401161.317 1845823.013),NaN,0.1,1.0,NaN,574130,0.000002
4,-15.925109,16.693707,5.0,POINT (401190.052 1845822.879),NaN,0.1,1.0,NaN,574130,0.000002
5,-15.924839,16.693707,6.0,POINT (401218.787 1845822.746),NaN,0.1,1.0,NaN,574130,0.000002
6,-15.924570,16.693707,7.0,POINT (401247.523 1845822.613),NaN,0.1,1.0,NaN,574130,0.000002
7,-15.924300,16.693707,8.0,POINT (401276.258 1845822.480),NaN,0.1,1.0,NaN,574130,0.000002
8,-15.924031,16.693707,9.0,POINT (401304.993 1845822.347),NaN,0.1,1.0,NaN,574130,0.000002
9,-15.922683,16.693707,10.0,POINT (401448.670 1845821.681),NaN,0.1,1.0,NaN,574130,0.000002


In [39]:
agriculture['spam_V'] = agriculture['spam_V'].fillna(0)
agriculture.head(30)

,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0,lc_count,spam_V
0,-15.990057,16.693707,1.0,POINT (394264.774 1845856.132),NaN,0.1,1.0,NaN,574130,0.000002
1,-15.989787,16.693707,2.0,POINT (394293.510 1845855.989),NaN,0.1,1.0,NaN,574130,0.000002
2,-15.989518,16.693707,3.0,POINT (394322.246 1845855.846),NaN,0.1,1.0,NaN,574130,0.000002
3,-15.925378,16.693707,4.0,POINT (401161.317 1845823.013),NaN,0.1,1.0,NaN,574130,0.000002
4,-15.925109,16.693707,5.0,POINT (401190.052 1845822.879),NaN,0.1,1.0,NaN,574130,0.000002
5,-15.924839,16.693707,6.0,POINT (401218.787 1845822.746),NaN,0.1,1.0,NaN,574130,0.000002
6,-15.924570,16.693707,7.0,POINT (401247.523 1845822.613),NaN,0.1,1.0,NaN,574130,0.000002
7,-15.924300,16.693707,8.0,POINT (401276.258 1845822.480),NaN,0.1,1.0,NaN,574130,0.000002
8,-15.924031,16.693707,9.0,POINT (401304.993 1845822.347),NaN,0.1,1.0,NaN,574130,0.000002
9,-15.922683,16.693707,10.0,POINT (401448.670 1845821.681),NaN,0.1,1.0,NaN,574130,0.000002


In [40]:
agriculture.tail(30)

,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0,lc_count,spam_V
2036,-13.625665,12.041980,0.1,None,NaN,2482.0,235934.0,2036.0,1,235934.0
2037,-13.542332,12.041980,0.1,None,NaN,2483.0,283561.0,2037.0,1,283561.0
2038,-13.458999,12.041980,0.1,None,NaN,2484.0,234852.0,2038.0,1,234852.0
2039,-13.286244,12.041983,0.1,None,NaN,2485.0,24589.0,2039.0,1,24589.0
2040,-13.209000,12.041980,0.1,None,NaN,2486.0,13736.0,2040.0,1,13736.0
2041,-13.125667,12.041980,0.1,None,NaN,2487.0,64784.0,2041.0,1,64784.0
2042,-13.042334,12.041980,0.1,None,NaN,2488.0,179684.0,2042.0,1,179684.0
2043,-12.959001,12.041980,0.1,None,NaN,2489.0,231312.0,2043.0,1,231312.0
2044,-12.875668,12.041980,0.1,None,NaN,2490.0,1007496.0,2044.0,1,1007496.0
2045,-12.792335,12.041980,0.1,None,NaN,2491.0,289588.0,2045.0,1,289588.0


In [41]:
agriculture.to_csv(os.path.join(out_pth, 'agriculture.csv')) # CSV circumvents some gdf issues later on.
# Note that it will take a few min to write to file.

In [42]:
# Origins and destinations must be in the WGS84 coordinate system to snap to graph.
agriculture = agriculture.to_crs("EPSG:4326")
agriculture.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Origins and destinations

Measure distance from origin/destination to nearest node and save to file.

In [43]:
#%% If starting new session, reload graph from file
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime.pickle")

In [ ]:
gn.example_edge(gTime)

In [46]:
# pandana_snap_c was giving an Index Error. Resetting the index didn't work.
# Re-loading seemed to resolve whatever the issue was.
agriculture = os.path.join(out_pth, "agriculture.csv")
agriculture = pd.read_csv(agriculture)
agriculture

crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(agriculture.x, agriculture.y)]
agriculture = GeoDataFrame(agriculture, crs=crs, geometry=geometry) 
agriculture.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7657065 entries, 0 to 7657064
Data columns (total 11 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Unnamed: 0    int64   
 1   x             float64 
 2   y             float64 
 3   ID_lc         float64 
 4   geometry      geometry
 5   index__spam   float64 
 6   ID_spam       float64 
 7   val           float64 
 8   Unnamed: 0.1  float64 
 9   lc_count      int64   
 10  spam_V        float64 
dtypes: float64(8), geometry(1), int64(2)
memory usage: 642.6 MB


In [47]:
agriculture.head()

,Unnamed: 0,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0.1,lc_count,spam_V
0,0,-15.990057,16.693707,1.0,POINT (-15.99006 16.69371),NaN,0.1,1.0,NaN,574130,0.000002
1,1,-15.989787,16.693707,2.0,POINT (-15.98979 16.69371),NaN,0.1,1.0,NaN,574130,0.000002
2,2,-15.989518,16.693707,3.0,POINT (-15.98952 16.69371),NaN,0.1,1.0,NaN,574130,0.000002
3,3,-15.925378,16.693707,4.0,POINT (-15.92538 16.69371),NaN,0.1,1.0,NaN,574130,0.000002
4,4,-15.925109,16.693707,5.0,POINT (-15.92511 16.69371),NaN,0.1,1.0,NaN,574130,0.000002


In [48]:
ag_snap = gn.pandana_snap_c(gTime, agriculture, source_crs = 'epsg:4326', target_crs = 'epsg:31028', add_dist_to_node_col = True)
ag_snap.to_csv('SEN-Cdrive/outputs/ag_snap.csv', index=True)
# Takes several minutes.
ag_snap

,Unnamed: 0,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0.1,lc_count,spam_V,NN,NN_dist
0,0,-15.990057,16.693707,1.0,POINT (-15.99006 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14607.791436
1,1,-15.989787,16.693707,2.0,POINT (-15.98979 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14628.808767
2,2,-15.989518,16.693707,3.0,POINT (-15.98952 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14649.852284
3,3,-15.925378,16.693707,4.0,POINT (-15.92538 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14029.850649
4,4,-15.925109,16.693707,5.0,POINT (-15.92511 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14018.013147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657060,2061,-11.459007,12.041980,0.1,POINT (-11.45901 12.04198),NaN,2507.0,411859.0,2061.0,1,411859.000000,2241013982,14274.586682
7657061,2062,-11.375674,12.041980,0.1,POINT (-11.37567 12.04198),NaN,2508.0,497200.0,2062.0,1,497200.000000,6662833878,13597.325940
7657062,2063,-11.292341,12.041980,0.1,POINT (-11.29234 12.04198),NaN,2509.0,217938.0,2063.0,1,217938.000000,6662833878,14994.531096
7657063,2064,-11.209008,12.041980,0.1,POINT (-11.20901 12.04198),NaN,2510.0,201222.0,2064.0,1,201222.000000,2217989971,20260.745415


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [49]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)

In [ ]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)

In [44]:
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime.pickle")

In [50]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
origins = list(ag_snap.NN.unique())

In [51]:
list_HDurban = list(HDurban_snap.NN.unique()) 
dests = list_HDurban

# If more than one set of destinations, you can combine the sets to go into a single OD matrix using this code:
# list_ssa = list(ssa_snap.NN.unique()) 
# destslist = list_HDurban + list_ssa
# dests = list(set(destslist))

In [52]:
len(origins) # 23108 unique nearest nodes (average of 5 origins per node).

15066

In [53]:
len(dests) # 314 unique nearest nodes. This is only 5 less than the original dataset.

57

In [54]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [55]:
gn.example_edge(gTime, 11)

(358284990, 5329792501, {'osmid': 59618174, 'ref': 'D 523', 'name': 'D 523', 'highway': 'unclassified', 'oneway': False, 'length': 73.22200000000001, 'geometry': <shapely.geometry.linestring.LineString object at 0x0000029C300F4040>, 'time': 5.271984000000001, 'mode': 'drive'})
(358284990, 5329792847, {'osmid': 178482063, 'highway': 'tertiary', 'oneway': False, 'length': 116.382, 'geometry': <shapely.geometry.linestring.LineString object at 0x0000029C300F4D00>, 'time': 6.98292, 'mode': 'drive'})
(358284990, 5217543372, {'osmid': 178482063, 'highway': 'tertiary', 'oneway': False, 'length': 187.25199999999998, 'geometry': <shapely.geometry.linestring.LineString object at 0x0000029B92E35D00>, 'time': 11.235119999999998, 'mode': 'drive'})
(358285037, 8835188674, {'osmid': 178470940, 'highway': 'tertiary', 'oneway': False, 'length': 27.743, 'time': 1.6645799999999997, 'mode': 'drive'})
(358285037, 8835188649, {'osmid': 178470940, 'highway': 'tertiary', 'oneway': False, 'length': 392.54100000

In [56]:
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 'time')
# This a few minutes.

In [57]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [58]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15066 entries, 5343951929 to 2217989971
Data columns (total 57 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   2119246531  15066 non-null  float64
 1   6029312286  15066 non-null  float64
 2   4998093094  15066 non-null  float64
 3   2201506815  15066 non-null  float64
 4   3474499811  15066 non-null  float64
 5   1697006012  15066 non-null  float64
 6   1901689169  15066 non-null  float64
 7   2846317227  15066 non-null  float64
 8   6040927878  15066 non-null  float64
 9   3449495495  15066 non-null  float64
 10  1918630462  15066 non-null  float64
 11  8972391475  15066 non-null  float64
 12  3418418812  15066 non-null  float64
 13  1983641803  15066 non-null  float64
 14  6014451367  15066 non-null  float64
 15  6027163238  15066 non-null  float64
 16  2833577864  15066 non-null  float64
 17  6040866461  15066 non-null  float64
 18  6045659374  15066 non-null  float64
 19  6038316595 

In [59]:
OD_df.tail()

,2119246531,6029312286,4998093094,2201506815,3474499811,1697006012,1901689169,2846317227,6040927878,3449495495,...,3631507838,3628686112,5427415265,6027616830,6027276894,6041227847,6048393788,299528744,8178289925,6026834850
2461703259,31435.934725,23059.716124,32484.330187,27571.544958,24731.930118,33919.281474,35232.961314,34223.674111,30466.625448,30632.241219,...,36170.763498,39520.433982,15343.231428,15361.182300,32739.313598,35057.780766,35092.750086,35380.783188,30842.866162,31159.375004
4933646636,33761.590891,25385.372290,34809.986353,29897.201124,27057.586284,36244.937640,37558.617480,36549.330277,32792.281614,32957.897385,...,38496.419664,41846.090148,17668.887594,17686.838466,35064.969764,37383.436932,37418.406252,37706.439354,33168.522328,33485.031170
4936068624,33224.679952,24848.461351,34273.075414,29360.290185,26520.675345,35708.026701,37021.706541,36012.419338,32255.370675,32420.986446,...,37959.508725,41309.179209,17131.976655,17149.927527,34528.058825,36846.525993,36881.495313,37169.528415,32631.611389,32948.120231
6662833878,37187.518189,28811.299588,38235.913651,33323.128422,30483.513582,39670.864938,40984.544778,39975.257575,36218.208912,36383.824683,...,41922.346962,45272.017446,21094.814892,21112.765764,38490.897062,40809.364230,40844.333550,41132.366652,36594.449626,36910.958468
2217989971,37041.964933,28665.746332,38090.360395,33177.575166,30337.960326,39525.311682,40838.991522,39829.704319,36072.655656,36238.271427,...,41776.793706,45126.464190,20949.261636,20967.212508,38345.343806,40663.810974,40698.780294,40986.813396,36448.896370,36765.405212


In [60]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'fromagriculture/OD.csv'))

In [61]:
OD_min.tail(20)

,2119246531,6029312286,4998093094,2201506815,3474499811,1697006012,1901689169,2846317227,6040927878,3449495495,...,3631507838,3628686112,5427415265,6027616830,6027276894,6041227847,6048393788,299528744,8178289925,6026834850
5072506110,520.509671,380.906027,537.982928,456.103175,408.776260,561.898783,583.793447,566.971994,504.354516,507.114779,...,599.423483,655.251325,252.297949,252.597130,542.232652,580.873771,581.456593,586.257145,510.625195,515.900342
4929139934,512.729786,373.126143,530.203044,448.323290,400.996376,554.118899,576.013563,559.192110,496.574632,499.334895,...,591.643599,647.471441,244.518065,244.817246,534.452768,573.093887,573.676709,578.477261,502.845310,508.120458
5086550496,547.389343,407.785699,564.862600,482.982847,435.655932,588.778455,610.673119,593.851666,531.234188,533.994451,...,626.303155,682.130997,279.177621,279.476802,569.112324,607.753443,608.336265,613.136817,537.504867,542.780014
3074157604,538.289308,398.685665,555.762566,473.882812,426.555898,579.678421,601.573084,584.751631,522.134153,524.894416,...,617.203121,673.030962,270.077586,270.376768,560.012289,598.653409,599.236231,604.036782,528.404832,533.679979
6662833882,619.492591,479.888948,636.965849,555.086095,507.759181,660.881704,682.776368,665.954915,603.337437,606.097700,...,698.406404,754.234246,351.280870,351.580051,641.215573,679.856692,680.439514,685.240066,609.608115,614.883263
3344553248,605.353852,465.750208,622.827110,540.947356,493.620442,646.742964,668.637628,651.816175,589.198697,591.958960,...,684.267665,740.095506,337.142130,337.441311,627.076833,665.717953,666.300775,671.101326,595.469376,600.744523
8630779288,92.105349,149.632941,366.031056,284.151302,286.637154,389.946911,411.841575,395.020121,332.402644,366.103102,...,452.863150,505.892846,310.916776,311.934982,370.280779,408.921899,409.504721,414.305273,363.825392,375.920381
2283013024,91.885519,149.413111,365.811226,283.931472,286.417325,389.727081,411.621745,394.800291,332.182814,365.883272,...,452.643321,505.673016,310.696946,311.715152,370.060950,408.702069,409.284891,414.085443,363.605562,375.700551
8609250930,90.363969,147.891561,364.289676,282.409922,284.895775,388.205531,410.100195,393.278742,330.661264,364.361722,...,451.121771,504.151467,309.175396,310.193602,368.539400,407.180519,407.763341,412.563893,362.084012,374.179001
2283463674,87.692927,145.220520,361.618634,279.738881,282.224733,385.534489,407.429153,390.607700,327.990222,361.690680,...,448.450729,501.480425,306.504354,307.522560,365.868358,404.509477,405.092299,409.892851,359.412971,371.507960


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [62]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD = os.path.join(out_pth, "fromagriculture/OD.csv")
OD = pd.read_csv(OD)

In [63]:
OD

,Unnamed: 0,2119246531,6029312286,4998093094,2201506815,3474499811,1697006012,1901689169,2846317227,6040927878,...,3631507838,3628686112,5427415265,6027616830,6027276894,6041227847,6048393788,299528744,8178289925,6026834850
0,5343951929,509.120314,490.899339,277.182209,316.481115,340.233093,241.755280,280.078827,224.844829,268.224856,...,155.200285,95.266157,495.844655,500.681138,278.636078,275.403918,274.658109,274.192740,244.583668,245.718518
1,3639403839,534.760083,516.539108,302.821978,342.120884,365.872862,267.395049,305.718596,250.484598,293.864625,...,180.840053,120.905926,521.484424,526.320907,304.275847,301.043687,300.297878,299.832509,270.223437,271.358287
2,2801436498,541.902768,523.681793,309.964662,349.263568,373.015547,274.537733,312.861281,257.627282,301.007310,...,187.982738,128.048610,528.627108,533.463591,311.418531,308.186371,307.440562,306.975193,277.366121,278.500971
3,4303444021,571.935935,553.714960,339.997830,379.296736,403.048714,304.570900,342.894448,287.660449,331.040477,...,218.015905,158.081778,558.660275,563.496758,341.451698,338.219538,337.473729,337.008361,307.399288,308.534138
4,3656617673,578.697165,560.476191,346.759060,386.057966,409.809945,311.332131,349.655678,294.421680,337.801708,...,224.777136,164.843008,565.421506,570.257989,348.212929,344.980769,344.234960,343.769591,314.160519,315.295369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15061,2461703259,523.932245,384.328602,541.405503,459.525749,412.198835,565.321358,587.216022,570.394569,507.777091,...,602.846058,658.673900,255.720524,256.019705,545.655227,584.296346,584.879168,589.679720,514.047769,519.322917
15062,4933646636,562.693182,423.089538,580.166439,498.286685,450.959771,604.082294,625.976958,609.155505,546.538027,...,641.606994,697.434836,294.481460,294.780641,584.416163,623.057282,623.640104,628.440656,552.808705,558.083853
15063,4936068624,553.744666,414.141023,571.217924,489.338170,442.011256,595.133778,617.028442,600.206989,537.589511,...,632.658479,688.486320,285.532944,285.832125,575.467647,614.108767,614.691589,619.492140,543.860190,549.135337
15064,6662833878,619.791970,480.188326,637.265228,555.385474,508.058560,661.181082,683.075746,666.254293,603.636815,...,698.705783,754.533624,351.580248,351.879429,641.514951,680.156071,680.738893,685.539444,609.907494,615.182641


In [64]:
OD.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) # Repeat for each OD set, if needed.

#### Find first, second, and third nearest destination for each origin node. 

In [65]:
fail_value = 999999999

In [66]:
# Since we only have one destination set, renaming the OD object for quick find/replace on this section of code.
OD_HDurban = OD

# Nearest
OD_HDurban["HDurban1"] = 0
sub = OD_HDurban.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_HDurban["HDurban1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
HDurban1 = OD_HDurban[['NN', 'HDurban1']] # Remove unnecessary OD values.
HDurban1.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban1.csv'))


# Second nearest
dupes = OD_HDurban.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_HDurban.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.

OD_HDurban["HDurban2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_HDurban["HDurban2"] = Dsub.min(axis=1) 
HDurban2 = OD_HDurban.loc[:,['NN', 'HDurban2']] 
HDurban2.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban2.csv'))


# Third nearest
dupes = OD_HDurban.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_HDurban.where(~dupes, fail_value)
 
OD_HDurban["HDurban3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_HDurban["HDurban3"] = Dsub.min(axis=1)
HDurban3 = OD_HDurban.loc[:,['NN', 'HDurban3']]
HDurban3.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban3.csv'))


# Combine and write to file
HDurban_all = OD_HDurban.loc[:,['NN', 'HDurban1', 'HDurban2', 'HDurban3']]
HDurban_all.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban_all.csv'))
HDurban_all.head()

,NN,HDurban1,HDurban2,HDurban3
0,5343951929,95.266157,95.613982,101.236310
1,3639403839,120.905926,121.253751,126.876079
2,2801436498,128.048610,128.396435,134.018763
3,4303444021,158.081778,158.429603,164.051930
4,3656617673,164.843008,165.190833,170.813161


### Join back to georeferenced _snap file.

In [72]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
ag_snap

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,x,y,ID_lc,geometry,index__spam,ID_spam,val,geom_UTM,geom_WGS84,lc_sum,spam_V,NN,NN_dist
0,0,0,-15.990057,16.693707,1.0,POINT (-15.9900569732419 16.6937073347074),NaN,NaN,0.0,NaN,NaN,NaN,0.0,5343951929,14607.791436
1,1,1,-15.989787,16.693707,2.0,POINT (-15.9897874786566 16.6937073347074),NaN,NaN,0.0,NaN,NaN,NaN,0.0,5343951929,14628.808767
2,2,2,-15.989518,16.693707,3.0,POINT (-15.9895179840714 16.6937073347074),NaN,NaN,0.0,NaN,NaN,NaN,0.0,5343951929,14649.852284
3,3,3,-15.925378,16.693707,4.0,POINT (-15.9253782727816 16.6937073347074),NaN,NaN,0.0,NaN,NaN,NaN,0.0,3639403839,14029.850649
4,4,4,-15.925109,16.693707,5.0,POINT (-15.9251087781964 16.6937073347074),NaN,NaN,0.0,NaN,NaN,NaN,0.0,3639403839,14018.013147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657060,7657060,2061,-11.459007,12.041980,NaN,POINT (-11.459007 12.04198),NaN,2507.0,411859.0,MULTIPOLYGON (((890065.2102591199 1329046.5648...,POINT (-11.45900694020285 12.041979526954448),0.0,inf,2241013982,14274.586682
7657061,7657061,2062,-11.375674,12.041980,NaN,POINT (-11.375674 12.04198),NaN,2508.0,497200.0,MULTIPOLYGON (((899153.5281643758 1329166.2613...,POINT (-11.375673926651558 12.041979526573865),0.0,inf,6662833878,13597.325940
7657062,7657062,2063,-11.292341,12.041980,NaN,POINT (-11.292341 12.04198),NaN,2509.0,217938.0,MULTIPOLYGON (((908242.6248140114 1329288.7282...,POINT (-11.292340913098172 12.041979526183944),0.0,inf,6662833878,14994.531096
7657063,7657063,2064,-11.209008,12.041980,NaN,POINT (-11.209008 12.04198),NaN,2510.0,201222.0,MULTIPOLYGON (((917332.5179180377 1329413.9667...,POINT (-11.209007899992315 12.041979525784605),0.0,inf,2217989971,20260.745415


In [32]:
ag_snap.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7657065 entries, 0 to 7657064
Data columns (total 13 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Unnamed: 0    int64   
 1   x             float64 
 2   y             float64 
 3   ID_lc         float64 
 4   geometry      geometry
 5   index__spam   float64 
 6   ID_spam       float64 
 7   val           float64 
 8   Unnamed: 0.1  float64 
 9   lc_sum        float64 
 10  spam_V        float64 
 11  NN            int64   
 12  NN_dist       float64 
dtypes: float64(10), geometry(1), int64(2)
memory usage: 759.4 MB


In [67]:
ag_to_HDurban = pd.merge(ag_snap, HDurban_all, on='NN',how='left')
ag_to_HDurban

,Unnamed: 0,x,y,ID_lc,geometry,index__spam,ID_spam,val,Unnamed: 0.1,lc_count,spam_V,NN,NN_dist,HDurban1,HDurban2,HDurban3
0,0,-15.990057,16.693707,1.0,POINT (-15.99006 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14607.791436,95.266157,95.613982,101.236310
1,1,-15.989787,16.693707,2.0,POINT (-15.98979 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14628.808767,95.266157,95.613982,101.236310
2,2,-15.989518,16.693707,3.0,POINT (-15.98952 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,5343951929,14649.852284,95.266157,95.613982,101.236310
3,3,-15.925378,16.693707,4.0,POINT (-15.92538 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14029.850649,120.905926,121.253751,126.876079
4,4,-15.925109,16.693707,5.0,POINT (-15.92511 16.69371),NaN,0.1,1.0,NaN,574130,0.000002,3639403839,14018.013147,120.905926,121.253751,126.876079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657060,2061,-11.459007,12.041980,0.1,POINT (-11.45901 12.04198),NaN,2507.0,411859.0,2061.0,1,411859.000000,2241013982,14274.586682,351.868825,353.256479,357.429453
7657061,2062,-11.375674,12.041980,0.1,POINT (-11.37567 12.04198),NaN,2508.0,497200.0,2062.0,1,497200.000000,6662833878,13597.325940,346.019620,347.407274,351.580248
7657062,2063,-11.292341,12.041980,0.1,POINT (-11.29234 12.04198),NaN,2509.0,217938.0,2063.0,1,217938.000000,6662833878,14994.531096,346.019620,347.407274,351.580248
7657063,2064,-11.209008,12.041980,0.1,POINT (-11.20901 12.04198),NaN,2510.0,201222.0,2064.0,1,201222.000000,2217989971,20260.745415,343.593733,344.981387,349.154361


In [68]:
ag_to_HDurban.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7657065 entries, 0 to 7657064
Data columns (total 16 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Unnamed: 0    int64   
 1   x             float64 
 2   y             float64 
 3   ID_lc         float64 
 4   geometry      geometry
 5   index__spam   float64 
 6   ID_spam       float64 
 7   val           float64 
 8   Unnamed: 0.1  float64 
 9   lc_count      int64   
 10  spam_V        float64 
 11  NN            int64   
 12  NN_dist       float64 
 13  HDurban1      float64 
 14  HDurban2      float64 
 15  HDurban3      float64 
dtypes: float64(12), geometry(1), int64(3)
memory usage: 993.1 MB


In [69]:
ag_to_HDurban = ag_to_HDurban.drop(columns=['Unnamed: 0.1', 'index__spam'])
ag_to_HDurban.rename(columns={'Unnamed: 0': 'ID_ag'}, inplace=True)
ag_to_HDurban.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7657065 entries, 0 to 7657064
Data columns (total 14 columns):
 #   Column    Dtype   
---  ------    -----   
 0   ID_ag     int64   
 1   x         float64 
 2   y         float64 
 3   ID_lc     float64 
 4   geometry  geometry
 5   ID_spam   float64 
 6   val       float64 
 7   lc_count  int64   
 8   spam_V    float64 
 9   NN        int64   
 10  NN_dist   float64 
 11  HDurban1  float64 
 12  HDurban2  float64 
 13  HDurban3  float64 
dtypes: float64(10), geometry(1), int64(3)
memory usage: 876.3 MB


In [70]:
ag_to_HDurban.to_csv(os.path.join(out_pth, 'fromagriculture/ag_to_HDurban.csv'))

### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml